In [1]:
import nbformat as nf
import nbconvert as nc
import os
import subprocess
from glob import glob
from tqdm import tqdm
from datetime import datetime
import shutil as sh

In [2]:
# Path to root of site
path_root = os.path.abspath('../../')

# Finds the ipynb files
files = glob('../final-notebooks/*/*/*.ipynb')

# A dictionary that runs the function with the value as input
run_keys = {'modified': lambda a: a.format(datetime.today())}

# Where to save the MD posts
save_path = '../../_posts/earth-analytics-python/'

In [3]:
for ifile in tqdm(files):
    # Find file names etc
    path = ifile.split('final-notebooks/')[-1]  # 
    path_tmp_file = os.path.basename(path)
    file_name = os.path.splitext(path_tmp_file)[0]
    path_folders = os.path.dirname(path)
    path_rel_to_root = os.path.dirname(ifile.split('final-notebooks')[1])

    # Read in notebook, pull meta cell
    ntbk = nf.read(ifile, nf.NO_CONVERT)
    meta = ntbk['cells'][0]
    if meta['cell_type'] == 'raw':

        # Only use lines with key: val pairs
        metalines = meta['source'].split('\n') 
        for ii, ln in enumerate(metalines):
            if '---' in ln:
                continue
            split = ln.split(': ')
            if len(split) < 2:
                continue
            if len(split) > 2:
                ln_type = split[0]
                ln_val = ': '.join(split[1:])
            else:
                ln_type, ln_val = split  
                
            # If we have a function for one of the keys, run it and replace the value
            if ln_type not in run_keys.keys():
                continue
            new_val = eval(run_keys[ln_type](ln_val))
            metalines[ii] = ': '.join([ln_type, new_val])
        
        # Replace the meta cell
        meta['source'] = '\n'.join(metalines)
        ntbk['cells'][0] = meta

    # Write the file
    path_save_tmp_file = os.path.join('./tmp', path_tmp_file)
    nf.write(ntbk, path_save_tmp_file)
    
    # Call nbconvert
    build_path = os.path.join(path_root, "_posts/course-materials/earth-analytics/{}".format(path_rel_to_root))
    build_call = '--FilesWriter.build_directory={}'.format(build_path)
    
    path_imgs_root = os.path.relpath(path_root, build_path)
    images_path = os.path.join(path_imgs_root, "images/course-materials/earth-analytics/{}".format(path_rel_to_root))
    images_call = '--NbConvertApp.output_files_dir={}'.format(images_path)


    subprocess.call(['jupyter', 'nbconvert',
                     '--to', 'markdown',
                     images_call, build_call, path_save_tmp_file])
    
    # Clean up
    os.remove(path_save_tmp_file)

100%|██████████| 30/30 [00:53<00:00,  1.90s/it]
